#Libraries and dataset

Tokenizerni yaratishni, kutubxonalarni o'rnatish, huggingfacega kirish va kerakli datasetni yuklash bilan boshlaymiz

**Dataset** - Tokenizer uchun `Common voice` dataasetining `Train` va `Validated` splitlarini qo'shib yuboramiz va natijada `130.000` qatordan uzunroq bo'lgan datasetga ega bo'lamiz

In [ ]:
!pip install datasets

In [4]:
from huggingface_hub import login
login('-')

In [ ]:
from datasets import load_dataset
dataset = load_dataset('mozilla-foundation/common_voice_17_0', 'uz', split='train+validated')

In [30]:
print(dataset)
print(dataset[0])

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
    num_rows: 134905
})
{'client_id': '2160561702bac0e2048d2dc79810c2d8a6e6942a6dcac881613c06c4e55bff89d8a701dd45bbb60db546539d4ffb267306404c307352c9db5b3fbf197e8be9ed', 'path': '/root/.cache/huggingface/datasets/downloads/extracted/0b52958eca3ab745b422e0bde22d2aaf511dc1ff783ac92f74513c6b0771cb62/uz_train_0/common_voice_uz_28907218.mp3', 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/0b52958eca3ab745b422e0bde22d2aaf511dc1ff783ac92f74513c6b0771cb62/uz_train_0/common_voice_uz_28907218.mp3', 'array': array([ 4.26325641e-14,  0.00000000e+00,  9.94759830e-14, ...,
       -5.12599363e-05, -8.97714926e-05, -6.41083534e-05]), 'sampling_rate': 48000}, 'sentence': 'Bugun ertalab Gyotenikiga taklifnoma oldim.', 'up_votes': 2, 'down_votes': 0, 'age': 'twenties', 'gender': 'male_masculine', 'accent': '', 'locale': 'uz', 's

#Getting corpus

Tokenizerni Iterator yordamida yaratamiz, Old tokenizer sifatida esa Facebookning `xlm-roberta-base` modelining tokenizeridan foydalanamiz

In [12]:
def corpus(dataset):
  for index in range(len(dataset), 5000):
    yield dataset[index,index+5000]['sentence']

In [13]:
from transformers import AutoTokenizer
old_tokenizer = AutoTokenizer.from_pretrained('FacebookAI/xlm-roberta-base')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [14]:
corpus = corpus(dataset)

#New Tokenizer

In [15]:
new_tokenizer = old_tokenizer.train_new_from_iterator(corpus, 120000)

In [18]:
new_tokenizer.save_pretrained("uz_tokenizer")

('uz_tokenizer/tokenizer_config.json',
 'uz_tokenizer/special_tokens_map.json',
 'uz_tokenizer/tokenizer.json')

#Saving

Huggingfaceda yangi repository yaratamiz va natijada olgan tokenizerimizni yuklaymiz

In [21]:
from huggingface_hub import HfApi, HfFolder
from transformers import AutoTokenizer
import os

HfFolder.save_token("-")

repo_id = "jamshidahmadov/uz_tokenizer"
tokenizer_path = "uz_tokenizer"

api = HfApi()
api.create_repo(repo_id=repo_id, repo_type="model", exist_ok=True)
api.upload_folder(folder_path=tokenizer_path, repo_id=repo_id, repo_type="model")

CommitInfo(commit_url='https://huggingface.co/jamshidahmadov/uz_tokenizer/commit/b050f805252f8a262cdc4c47e18f08ab83180c2b', commit_message='Upload folder using huggingface_hub', commit_description='', oid='b050f805252f8a262cdc4c47e18f08ab83180c2b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jamshidahmadov/uz_tokenizer', endpoint='https://huggingface.co', repo_type='model', repo_id='jamshidahmadov/uz_tokenizer'), pr_revision=None, pr_num=None)

#Testing

Keling, Yangi Tokenizerni sinab ko'ramiz

In [29]:
tokenizer = AutoTokenizer.from_pretrained("jamshidahmadov/uz_tokenizer")
test_text = "Bu text eski tokenizer va yangi tokenizerning o'rtasidagi farqni solishtirish uchun"

print(old_tokenizer.tokenize(test_text))
print(tokenizer.tokenize(test_text))

['▁Bu', '▁text', '▁eski', '▁to', 'ken', 'izer', '▁va', '▁yangi', '▁to', 'ken', 'izer', 'ning', '▁o', "'", 'rtasidagi', '▁farq', 'ni', '▁so', 'lish', 'tirish', '▁uchun']
['▁Bu', '▁text', '▁eski', '▁tokenizer', '▁va', '▁yangi', '▁tokenizerning', "▁o'rtasidagi", '▁farqni', '▁solishtirish', '▁uchun']


Natija ancha yaxshi. Nafaqat NER, balki kelajakdagi boshqa  modellarni tuzish uchun ham  ushbu tokenizerni ishlatishimiz mumkin